Assignment 4
Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from assets/wikipedia_data.html and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in assets/nfl.csv), MLB (baseball, in assets/mlb.csv), NBA (basketball, in assets/nba.csv or NHL (hockey, in assets/nhl.csv). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: what is the win/loss ratio's correlation with the population of the city it is in? Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with pearsonr, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data from year 2018 for your analysis -- this is important!

Notes
    1.Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this     assignment.
    2.I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
    3.It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
    4.There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NHL using 2018 data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#print(cities)
cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)
cities['NHL'] = cities['NHL'].str.replace(r'\[.*\]','')
#print(cities)

def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    team = cities['NHL'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z][a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
    #some of the states have 3 teams from that area so the regex was done thrice.
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area'])
    team = team.drop(columns = ['variable'])
    team = team.replace("", np.nan).replace("—",np.nan).dropna().reset_index()
    team = team.rename(columns = {'value':'team_name'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area')
    team = team.iloc[:,1:4]
    team['Population'] = team['Population'].astype(int)
    team['team_name'] = team['team_name'].str.replace('[\w.]*\ ', '')
    #print(team)
    
    
    points_table = pd.read_csv('assets/nhl.csv')
    points_table =  points_table[points_table['year'] == 2018]
    points_table['team_name'] = points_table['team'].str.replace(r'\*', "")
    points_table = points_table[['team_name', 'W', 'L']]
    points_table = points_table.drop([0,9,18,26])
    points_table['team_name'] = points_table['team_name'].str.replace('[\w.]*\ ', '')
    points_table['W'] = points_table['W'].astype(int)
    points_table['L'] = points_table['L'].astype(int)
    points_table['points_ratio'] = points_table['W']/(points_table['W'] + points_table['L'])
    #print(points_table)
    
    final_df = pd.merge(team, points_table, how = 'outer', on = 'team_name')
    final_df = final_df.groupby('Metropolitan area').agg({'points_ratio':np.mean,'Population':np.mean})
    #print(final_df.reset_index())
    
    population_by_region = final_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = final_df['points_ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

In [2]:
nhl_correlation()

0.012486162921209881

Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.



In [117]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)
cities['NBA'] = cities['NBA'].str.replace(r'\[.*\]','')
#print(cities)

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    team = cities['NBA'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    #some of the states have 3 teams from that area so the regex was done thrice.
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area'])
    team = team.drop(columns = ['variable'])
    team = team.replace("", np.nan).replace("—",np.nan).dropna().reset_index()
    team = team.rename(columns = {'value':'team_name'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area')
    team = team.iloc[:,1:4]
    team['Population'] = team['Population'].astype(int)
    team['team_name'] = team['team_name'].str.replace('[\w.]*\ ', '')
    team['team_name'] = team['team_name'].str.replace('76ers', 'ers')
    team['team_name'] = team['team_name'].str.strip()
    #print(team)
    
    points_table = pd.read_csv('assets/nba.csv')
    points_table =  points_table[points_table['year'] == 2018]
    points_table['team_name'] = points_table['team'].str.replace(r'[\*]', '')
    points_table['team_name'] = points_table['team_name'].str.replace(r'[\(\d*\)]', '')
    #points_table['team_name'] = points_table['team_name'].str.replace(r'[\xa0]', '')
    points_table = points_table[['team_name', 'W/L%']]
    #print(points_table)
    #points_table = points_table.drop([0,9,18,26])
    points_table['team_name'] = points_table['team_name'].str.replace('[\w.]*\ ', '')
    points_table['W/L%'] = points_table['W/L%'].astype(float)
    points_table['team_name'] = points_table['team_name'].str.strip()
    #points_table['L'] = points_table['L'].astype(int)
    #points_table['points_ratio'] = points_table['W']/(points_table['W'] + points_table['L'])
    #print(points_table)
    
    final_df = pd.merge(team, points_table, how = 'outer', on = 'team_name')
    final_df = final_df.groupby('Metropolitan area').agg({'W/L%':np.mean,'Population':np.mean})
    #print(final_df)
    
    population_by_region = final_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = final_df['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    corr, pval =  stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

In [118]:
print(nba_correlation())

-0.17636350642182935


Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data.

In [195]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)
cities['MLB'] = cities['MLB'].str.replace(r'\[.*\]','')
#print(cities)

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    team = cities['MLB'].str.extract('([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z][a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)([A-Z]?[a-z]*\ [A-Z]?[a-z]*|[A-Z]?[a-z]*)')
    #some of the states have 3 teams from that area so the regex was done thrice.
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area'])
    team = team.drop(columns = ['variable'])
    team = team.replace("", np.nan).replace("—",np.nan).dropna().reset_index()
    team = team.rename(columns = {'value':'team_name'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area')
    team = team.iloc[:,1:4]
    team['Population'] = team['Population'].astype(int)
    team['team_name'] = team['team_name'].str.replace('\ Sox', 'Sox')
    team['team_name'] = team['team_name'].str.replace('[\w.]*\ ', '')
    #print(team)
    
    points_table = pd.read_csv('assets/mlb.csv')
    points_table =  points_table[points_table['year'] == 2018]
    points_table['team_name'] = points_table['team'].str.replace(r'\*', "")
    points_table = points_table[['team_name', 'W-L%']]
    #points_table = points_table.drop([0,9,18,26])
    points_table['team_name'] = points_table['team_name'].str.replace('\ Sox', 'Sox')
    points_table['team_name'] = points_table['team_name'].str.replace('[\w.]*\ ', '')
    points_table['W-L%'] = points_table['W-L%'].astype(float)
    #points_table['L'] = points_table['L'].astype(int)
    #points_table['points_ratio'] = points_table['W']/(points_table['W'] + points_table['L'])
    #print(points_table)
    
    final_df = pd.merge(team, points_table, how = 'outer', on = 'team_name')
    final_df = final_df.groupby('Metropolitan area').agg({'W-L%':np.mean,'Population':np.mean})
    #print(final_df)
    
    population_by_region = final_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = final_df['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

In [196]:
print(mlb_correlation())

0.15003737475409498


Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NFL using 2018 data.

In [166]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r'\[.*\]','')
#print(cities)

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    team = cities['NFL'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    #some of the states have 3 teams from that area so the regex was done thrice.
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area'])
    team = team.drop(columns = ['variable'])
    team = team.replace("", np.nan).replace("—",np.nan).dropna().reset_index()
    team = team.rename(columns = {'value':'team_name'})
    team = pd.merge(team, cities, how='left', on='Metropolitan area')
    team = team.iloc[:,1:4]
    team['Population'] = team['Population'].astype(int)
    team['team_name'] = team['team_name'].str.replace('[\w.]*\', '')
    team['team_name'] = team['team_name'].str.strip()
    #print(team)
    
    points_table = pd.read_csv('assets/nfl.csv')
    points_table =  points_table[points_table['year'] == 2018]
    points_table['team_name'] = points_table['team'].str.replace(r'[\*\+]', "")
    points_table = points_table[['team_name', 'W-L%']]
    #print(points_table)
    points_table = points_table.drop([0,5,10,15,20,25,30,35])
    points_table['team_name'] = points_table['team_name'].str.replace('[\w.]*\ ', '')
    points_table['W-L%'] = points_table['W-L%'].astype(float)
    points_table['team_name'] = points_table['team_name'].str.strip()
    #points_table['L'] = points_table['L'].astype(int)
    #points_table['points_ratio'] = points_table['W']/(points_table['W'] + points_table['L'])
    #print(points_table)
    
    final_df = pd.merge(team, points_table, how = 'outer', on = 'team_name')
    final_df = final_df.groupby('Metropolitan area').agg({'W-L%':np.mean,'Population':np.mean})
    #print(final_df)
    
    population_by_region = final_df['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = final_df['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

In [167]:
print(nfl_correlation())

0.004282141436393022


Question 5
In this question I would like you to explore the hypothesis that given that an area has two sports teams in different sports, those teams will perform the same within their respective sports. How I would like to see this explored is with a series of paired t-tests (so use ttest_rel) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values